# Collect and Clean data

I collect data from 
1. COMPUSTAT
2. U.S Energy Information Administration (EIA)
3. Regulatory Researc Associates (RRA)

I clean the data, merge and preprocess it for analysis

In [ ]:
import pandas as pd
import numpy as np

import datetime
import re



## Import datasets
There are three data different datasets that I use. 
1. df contains data on fundamentals (balance sheet etc)
2. dff contains utility specific information 
3. dfff contains utility credit ratings

In [ ]:
df=pd.read_csv('.../data/utility_fundamentals_Qnew.csv',low_memory=False)

In [ ]:
dff=pd.read_csv('.../data/utility_specific_compustat_quarterly_final.csv',low_memory=False)

In [ ]:
dfff=pd.read_csv('.../data/utility_creditratings.csv',low_memory=False)

In [ ]:
print "Dataset has", str(df.shape[0]) + " rows and",str(df.shape[1]) + " columns"
print "Dataset has", str(dff.shape[0]) + " rows and",str(dff.shape[1]) + " columns"
print "Dataset has", str(dfff.shape[0]) + " rows and",str(dff.shape[1]) + " columns"

In [ ]:
print "There are", str(len(df.tic.unique())) + " companies in the fundamental dataset"
print "There are", str(len(dff.tic.unique())) + " companies in the utility specific dataset"
print "There are", str(len(dfff.tic.unique())) + " companies in the utility credit ratings dataset"



## Where are companies Incorporated?
I am only interested in US companies 

In [ ]:
np.sort(df.fic.unique()) #fic is the foreign incorporation code

In [ ]:
print "The number of firms incorporated outside the USA:", len(df[df.fic != 'USA']['conm'].unique())
df=df[df.fic=='USA'] # select only US firms

In [ ]:
print "There are", str(len(df.tic.unique())) + " companies REMAINING in the fundamental dataset"

## Firm ownership structure
I'm interested in publicly traded firms and their subsidiaries which implies firms that have 0 and 1 as the stko variable

In [ ]:
np.sort(df.stko.unique())

In [ ]:
print len(df[df.stko>1]['conm'].unique()) #number of companies not publicly traded or subsidiaries

### Select public traded and subsidiaries

In [ ]:
df=df[(df.stko==0) | (df.stko==1)] #0 is for public traded and 1 is for their subsidiaries

In [ ]:
print "There are", str(len(df[df.stko==1]['tic'].unique())) + " subsidaries in the data."

In [ ]:
df=df[df.fyearq>2001] #remove 2001 from the sample

In [ ]:
print "The number of public traded firms/subsidiaries incorporated in the USA:", len(df['conm'].unique())

### Check for columns that have no values (empty) and remove them

In [ ]:
def column_check(data):
    useless=[]
    useful=[] # hold columns that are not entirely null
    for row in list(data.columns):
        try:#if the # empty rows==total # of rows, this means all rows are empty
            if pd.isnull(data[row]).value_counts()[1]==data.shape[0]:
                useless.append(row)
            else:
                useful.append(row)
        except KeyError:
            useful.append(row)
    return useful

In [ ]:
df=df[column_check(df)]

In [ ]:
df.shape #93 columns were completely missing

## Handle date variables

In [ ]:
df['date']=pd.to_datetime(df['datadate'])
dff['date']=pd.to_datetime(dff.date)
dfff['date']=pd.to_datetime(dfff.datadate)
df['year']=df['date'].dt.year #create a year variable

## Merge the dataset

In [ ]:
match=list(set(dff.columns) & set(df.columns)) # create the common list of variables to match the data with

In [ ]:
df=df.sort_values(['tic','date'])
dff=dff.sort_values(['tic','date'])
dfff=dfff.sort_values(['tic','date'])

In [ ]:
df=pd.merge(df,dff,on=match,how='left')
print df.shape

In [ ]:
new_match=list(set(df.columns)& set(dfff.columns))

In [ ]:
df=pd.merge(df,dfff,on=new_match, how='left')

In [ ]:
df.shape

## Issues with capx and other variables
Some variables are given as year to date. To get the actual quarterly equivalent, one must substract from the lagged value
Steps
1. First group the data by ticker(company) and year
2. Subtract lagged value from actual value(year to date)

In [ ]:
grouped=df.groupby(['tic','fyearq'])

In [ ]:
new=[] ## for every tic and fiscal year, 
for group in grouped.groups.keys():
    DF=grouped.get_group(group)
    DF['capx']=DF['capxy']-DF['capxy'].shift(1)
    DF['plant_gross_add']=DF['uptacy']-DF['uptacy'].shift(1) #Utility Plant - Gross Additions
    DF['stock_sale']=DF['sstky']-DF['sstky'].shift(1)
    DF['stock_purchase']=DF['prstkcy']-DF['prstkcy'].shift(1)
    DF['cash_dividends']=DF['dvy']-DF['dvy'].shift(1)
    DF['LT_debt_issuance']=DF['dltisy']-DF['dltisy'].shift(1)
    DF['LT_debt_reduction']=DF['dltry']-DF['dltry'].shift(1)
    DF['NOCF']=DF['oancfy']-DF['oancfy'].shift(1)
    DF['NFCF']=DF['fincfy']-DF['fincfy'].shift(1)
    DF['NICF']=DF['ivncfy']-DF['ivncfy'].shift(1)
    DF['property_sale']=DF['sppey']-DF['sppey'].shift(1)
    DF['investment_sale']=DF['sivy']-DF['sivy'].shift(1)
    DF['other_cash']=DF['uoisy']-DF['uoisy'].shift(1)
    DF['asset_growth']=DF['atq']-DF['atq'].shift(1)
    DF['yearly_total_sales']=DF['saleq'].sum()
    DF['yearly_total_assets']=DF['atq'].sum()
    new.append(DF)
    

In [ ]:
new_df=pd.concat(new)

In [ ]:
new_df=new_df.reset_index()

In [ ]:
def if_first_null(data,x,y):
    dd=[]
    ### check if first value is null, if so replace it with the year to date
    for i in range(len(data)):
        if pd.isnull(data[x])[i]==True:
            dd.append(data[y][i])
        else:
            dd.append(data[x][i])
    return dd

In [ ]:
new_df['capx']=if_first_null(new_df,'capx','capxy')
new_df['plant_gross_add']=if_first_null(new_df,'plant_gross_add','uptacy')
new_df['stock_sale']=if_first_null(new_df,'stock_sale','sstky')
new_df['stock_purchase']=if_first_null(new_df,'stock_purchase','prstkcy')
new_df['LT_debt_issuance']=if_first_null(new_df,'LT_debt_issuance','dltisy')
new_df['LT_debt_reduction']=if_first_null(new_df,'LT_debt_reduction','dltry')
new_df['other_cash']=if_first_null(new_df,'other_cash','uoisy')
new_df['property_sale']=if_first_null(new_df,'property_sale','sppey')
new_df['investment_sale']=if_first_null(new_df,'investment_sale','sivy')
new_df['cash_dividends']=if_first_null(new_df,'cash_dividends','dvy')
new_df['NOCF']=if_first_null(new_df,'NOCF','oancfy')
new_df['NFCF']=if_first_null(new_df,'NFCF','fincfy')
new_df['NICF']=if_first_null(new_df,'NICF','ivncfy')

In [ ]:
df=new_df

In [ ]:
print "The number of unique companies in the dataset: ", len(df.tic.unique())

## States and regulation

In [ ]:
print 'There are', str(len(df.state.unique())) + ' states in the sample'

In [ ]:
states=list(np.sort(df.state.unique()))

## Regulation data from U.S Energy Information Administration
deregulation and retail_choice values were obtained based on the sorted list of states and matching with relevant data from EIA on deregulation and retail_choice

In [ ]:
deregulation=['NO','suspended','suspended','suspended','NO','YES','YES','YES','NO','NO','NO','NO','NO','YES','NO','NO','NO','NO','YES','YES','YES','YES','NO','NO','NO','suspended','NO','YES','YES','suspended','suspended','YES','YES','NO','YES','YES','NO','NO','YES','suspended','NO','NO','NO','NO']
retail_choice=['NO','NO','NO','suspended','NO','YES','YES','YES','NO','NO','NO','NO','NO','YES','NO','NO','NO','NO','YES','YES','YES','YES','NO','NO','NO','NO','NO','YES','YES','NO','NO','YES','YES','NO','YES','YES','NO','NO','YES','suspended','NO','NO','NO','NO']

In [ ]:
S=pd.DataFrame(states)
S.rename(columns={0:'state'},inplace=True) #change column 0 to state

In [ ]:
S['deregulation']=deregulation
S['retail_choice']=retail_choice
S['deregulation']=S['deregulation'].str.upper()
S['retail_choice']=S['retail_choice'].str.upper()

In [ ]:
S.deregulation.value_counts()

In [ ]:
df=pd.merge(df,S,left_on='state',right_on='state',how='outer')

### Adjust states that were deregulated then moved to suspend

In [ ]:
dd=[]
for index,row in df.iterrows():
    if row['date']<datetime.datetime(2007,2,1) and row['state']=='VA':
        dd.append('YES')
    elif row['date'] < datetime.datetime(2004,10,1) and row['state']=='AZ':
        dd.append('YES')
    else:
        dd.append(row['deregulation'])

In [ ]:
df['deregulation']=dd 

## Asset value variables

In [ ]:
df['log_asset']=np.log(df['atq'])

In [ ]:
df['market_value']=df['prccq'] * df['cshoq'] 

In [ ]:
df['capx_asset']=(df['capx']/df['atq']) *100 # capital expenditure to asset
df['log_capx']=np.log(df.capx)
df['log_capx_asset']=np.log(df.capx_asset)
df['ppentq_asset']=(df['ppentq']/df['atq']) *100 
df['ppegtq_asset']=(df['ppegtq']/df['atq']) *100
df['RD_asset']=df.xrdq/df.atq # R&D

In [ ]:
df['plant_add_asset']=(df['plant_gross_add']/df['atq']) * 100
df['log_plant_asset']=np.log(df.plant_add_asset)
df['log_plant_add']=np.log(df.plant_gross_add)
df['capital_intensity']=(df['ppegtq']/df.atq) * 100 #gross plants

In [ ]:
df['inventory_asset']=(df['invtq']/df.atq) * 100
df['net_workingcap_asset']=((df.actq-df.lctq-df.cheq)/df.atq) *100

## Equity

In [ ]:
df['stock_sale_asset']=(df.stock_sale/df.atq) * 100

In [ ]:
df['net_equity_issuance']=df['stock_sale']-df['stock_purchase']

In [ ]:
df['net_equity_asset']=(df.net_equity_issuance/df.atq)*100


In [ ]:
df['EED']=df['net_equity_issuance']/df['capx'] #external equity dependence

## Returns

In [ ]:
df['ROE']=(df.uniamiq-df.dvpq)/df.ceqq ## return on equity

In [ ]:
df['ROA']=df.uniamiq/df['atq'] #net income before extraordinary items

## Debt

In [ ]:
df['total_debt_q']=df['dlcq']+df['dlttq'] # Total_debt== short-term debt(COMPUSTAT: DLCQ)+ long-term debt (DLTTQ).
df['debt_asset']=(df['total_debt_q']/df['atq']) * 100
df['ST_debt_asset']=(df['dlcq']/df.atq) * 100
df['LT_debt_asset']=(df['dlttq']/df.atq) * 100
df['leverage_ratio']=(df.total_debt_q/(df.seqq+df.dlttq))*100
df['debt_equity']=df.total_debt_q/df.seqq


In [ ]:
df['LT_debt_issuance_asset']=(df.LT_debt_issuance/df.atq)*100
df['LT_debt_reduction_asset']=(df.LT_debt_reduction/df.atq)*100
df['net_debt']=df['LT_debt_issuance']-df['LT_debt_reduction']
df['net_debt_asset']=(df['net_debt']/df['atq']) * 100

## Cash

In [ ]:
df['cash_asset']=(df['cheq']/df['atq']) * 100 #cash and short term investments/total assets
df['other_cash_asset']=(df['other_cash']/df.atq)*100

In [ ]:
df['cash_dividends_asset']=(df['cash_dividends']/df.atq)*100

In [ ]:
df['cash_flow']=df['ibq'] + df['dpq'] +df['ppentq'] #Income Before Extraordinary Items plus Depreciation and Amortization
df['cash_flow_asset']=(df['cash_flow']/df['atq']) * 100

In [ ]:
df['sale_asset']=(df.saleq/df.atq) * 100

In [ ]:
df['external_finance']=((df.plant_gross_add-df.cash_flow)/df.plant_gross_add) * 100


## Sources of funds

In [ ]:
df['investment_sale_asset']=(df['investment_sale']/df['atq']) * 100
df['property_sale_asset']=(df['property_sale']/df['atq']) * 100
df['NOCF_asset']=(df['NOCF']/df['atq']) * 100
df['NICF_asset']=(df['NICF']/df['atq']) * 100
df['NFCF_asset']=(df['NFCF']/df['atq']) * 100


## Earnings, interest, profit

In [ ]:
df['ebitda']=df['revtq']-df['xoprq'] + df['dpq'] #EBITDA = Revenue (REVTQ) – Operating expenses (XOPRQ) + depreciation and amortization (DPQ)
df['ebitda_asset']=(df['ebitda']/df['atq'])*100

In [ ]:
df['retainedearnings_asset']=(df['req']/df.atq)*100

In [ ]:
df['interest_coverage_ratio']=df['ebitda']/df['xintq']
df['interest_coverage_ratio_asset']=df['interest_coverage_ratio']/df['atq']

In [ ]:
df['profit']=(df['piq']/df['atq']) * 100

In [ ]:
df['yearly_total_sales_asset']=df['yearly_total_sales']/df['yearly_total_assets']

In [ ]:
df['stock_purchase_asset']=(df.stock_purchase/df.atq)*100

In [ ]:
df['payout_asset']=df['cash_dividends_asset']+ df['stock_purchase_asset']

In [ ]:
df=df.sort_values(['tic','date'])

In [ ]:
df=df.reset_index()

In [ ]:
del df['level_0']

In [ ]:
## Create lagged variables
grouped=df.groupby(['tic',])

In [ ]:
new=[] ## for every tic and fiscal year, 
for group in grouped.groups.keys():
    DF=grouped.get_group(group)
    DF['lag_capx']=DF['capx'].shift(1)
    DF['lag_plant_gross_add']=DF['plant_gross_add'].shift(1) #Utility Plant - Gross Additions
    #DF['lag_stock_sale']=DF['stock_sale'].shift(1)
    #DF['lag_stock_purchase']=DF['stock_purchase'].shift(1)
    DF['lag_asset_growth']=DF['asset_growth'].shift(1)
    DF['lag_cash_flow_asset']=DF['cash_flow_asset'].shift(1)
    DF['lag_cash_asset']=DF['cash_asset'].shift(1)
    DF['lag_debt_asset']=DF['debt_asset'].shift(1)
    DF['lag_LT_debt_asset']=DF['LT_debt_asset'].shift(1)
    DF['lag_ST_debt_asset']=DF['ST_debt_asset'].shift(1)
    DF['four_lags_LT_debt_asset']=DF['LT_debt_asset'].shift(4)
    DF['four_lags_ST_debt_asset']=DF['ST_debt_asset'].shift(4)
    DF['four_lags_cash_asset']=DF['cash_asset'].shift(4)
    DF['four_lags_debt_asset']=DF['debt_asset'].shift(4)
    DF['four_lags_cash_flow_asset']=DF['cash_flow_asset'].shift(4)
    DF['lag_sale_asset']=DF['sale_asset'].shift(1)
    DF['four_lags_sale_asset']=DF['sale_asset'].shift(4)
    DF['lag_yearly_total_sales_asset']=DF['yearly_total_sales_asset'].shift(4)
    DF['lag_yearly_total_assets']=DF['yearly_total_assets'].shift(4)
    #DF['net_debt_issuance']=DF['total_debt_q']-DF['total_debt_q'].shift(1)
    new.append(DF)
df=pd.concat(new)


In [ ]:
df=pd.concat(new)

## Sales growth

In [ ]:
df['sale_growth']=((df['sale_asset']-df['four_lags_sale_asset'])/df.four_lags_sale_asset) *100

In [ ]:
df['yearly_sale_growth']=(df['yearly_total_sales_asset']-df['lag_yearly_total_sales_asset'])/df['lag_yearly_total_sales_asset']

In [ ]:
df[df.saleq<=0]['tic'].unique()

In [ ]:
df=df[df.saleq>0]

## Create dummy variables

In [ ]:
quarter_dummies=pd.get_dummies(df.fqtr,prefix='quarter')

In [ ]:
df=pd.concat([df,quarter_dummies],axis=1)

In [ ]:
remove_list=list(df[pd.isnull(df.plant_add_asset)]['tic'].unique()) # companies that have some missing data on capx
print "There are " + str(len(remove_list)) + " companies with missing values on investment"

In [ ]:
dd=[]
for row in df['tic']:
    if row in remove_list:
        dd.append(0)
    else:
        dd.append(1)

In [ ]:
df['remove']=dd

In [ ]:
dd=[]
for index,row in df.iterrows():
    if row['fyearq']==2006 and row['fqtr']==2:
        dd.append(1)
    else:
        dd.append(0)
df['year_before']=dd

In [ ]:
tercile_group=df.groupby('year_before')
NEW=[] ## 
for group in tercile_group.groups.keys():
    DFF=tercile_group.get_group(group)
    DFF['debt_tercile']=pd.qcut(DFF.debt_asset,3,labels=["low_debt","medium_debt","high_debt"])
    DFF['cash_tercile']=pd.qcut(DFF.cash_asset,3,labels=["low_cash","medium_cash","high_cash"])
    DFF['asset_tercile']=pd.qcut(DFF.atq,3,labels=["low_asset","medium_asset","high_asset"])
    NEW.append(DFF)
df=pd.concat(NEW)

In [ ]:
dd=[]
for index,row in df.iterrows():
    if row['year_before']==1:
        dd.append(row['debt_tercile'])
    else:
        dd.append(None)
df['debt_tercile']=dd

dd=[]
for index,row in df.iterrows():
    if row['year_before']==1:
        dd.append(row['cash_tercile'])
    else:
        dd.append(None)
df['cash_tercile']=dd

dd=[]
for index,row in df.iterrows():
    if row['year_before']==1:
        dd.append(row['asset_tercile'])
    else:
        dd.append(None)
df['asset_tercile']=dd

In [ ]:
median_group=df.groupby('year_before')
NEW=[] ## 
for group in median_group.groups.keys():
    DFF=median_group.get_group(group)
    DFF['debt_median']=pd.qcut(DFF.debt_asset,2,labels=["LD","HD"])
    DFF['cash_median']=pd.qcut(DFF.cash_asset,2,labels=["LC","HC"])
    DFF['asset_median']=pd.qcut(DFF.atq,2,labels=["LA","HA"])
    NEW.append(DFF)
df=pd.concat(NEW)

In [ ]:
def year_before_median(data,finVar):
    '''This function takes every value of the particular finVar 
    other than year before and sets it to 0'''
    dd=[]
    for index,row in data.iterrows():
        if row['year_before']==1:
            dd.append(row[finVar])
        else:
            dd.append(None)
    return dd

In [ ]:
df['debt_median']=year_before_median(df,'debt_median')
df['cash_median']=year_before_median(df,'cash_median')
df['asset_median']=year_before_median(df,'asset_median')


In [ ]:
df=df.sort_values(['tic','fyearq','fqtr'])
df['debt_tercile']=df['debt_tercile'].fillna(method='ffill')
df['cash_tercile']=df['cash_tercile'].fillna(method='ffill')
df['asset_tercile']=df['asset_tercile'].fillna(method='ffill')

In [ ]:
df=df.sort_values(['tic','fyearq','fqtr'])
df['debt_median']=df['debt_median'].fillna(method='ffill')
df['cash_median']=df['cash_median'].fillna(method='ffill')
df['asset_median']=df['asset_median'].fillna(method='ffill')

In [ ]:
## define variables like year_cash, year_debt etc
df['year_cash']=[df.cash_asset if x==1 else None for x in df['year_before']]

In [ ]:
def variable_yearbefore(data,yearV,finV):
    '''This function takes the value of a financial variable from the year before'''
    dd=[]
    for index,row in data.iterrows():
        if row[yearV]==1:
            dd.append(row[finV])
        else:
            dd.append(None)
    return dd

In [ ]:
df['year_cash']=variable_yearbefore(df,'year_before','cash_asset')
df['year_debt']=variable_yearbefore(df,'year_before','debt_asset')
df['year_ST_debt']=variable_yearbefore(df,'year_before','LT_debt_asset')
df['year_LT_debt']=variable_yearbefore(df,'year_before','ST_debt_asset')

In [ ]:
df=df.sort_values(['tic','fyearq','fqtr'])
df['year_cash']=df['year_cash'].fillna(method='ffill')
df['year_debt']=df['year_debt'].fillna(method='ffill')
df['year_LT_debt']=df['year_LT_debt'].fillna(method='ffill')
df['year_ST_debt']=df['year_ST_debt'].fillna(method='ffill')

In [ ]:
dd=[]
for index,row in df.iterrows():
    if row['fyearq']<=2006:
        dd.append(0)
    elif row['fyearq']==2007 and row['fqtr']<=2:
        dd.append(0)
    else:
        dd.append(1)
df['post']=dd

## Industry component
Need to figure out what sector of the electric power industry a company operates in

In [ ]:
dd=[]
for row in df['busdesc']:
    if 'vertical' in row:
        dd.append('vertical')
    elif 'generat' in row and 'transmi' in row and 'distribu' in row:
        dd.append('vertical')
    elif 'generat' in row and 'deliv' in row:
        dd.append('vertical')
    elif 'generat' in row and 'suppl' in row:
        dd.append('vertical')
    elif 'generat' in row and 'transmi' not in row and 'distribu' not in row and 'deliv' not in row:
        dd.append('generation')
    elif 'generat' not in row and 'transmi' in row and 'distribu' not in row:
        dd.append('transmission')
    elif 'generat' not in row and 'transmi' not in row and 'distribu' in row:
        dd.append('distribution')
    else:
        dd.append('T&D')
df['utility_type']=dd
df['segment']=['T&D' if x == 'transmission' or x =='distribution' or x=='T&D' else x for x in df['utility_type']]

In [ ]:
segment_dummy=pd.get_dummies(df.segment)
debt_dummy=pd.get_dummies(df.debt_tercile)
cash_dummy=pd.get_dummies(df.cash_tercile)
asset_dummy=pd.get_dummies(df.asset_tercile)
new_debt_dummy=pd.get_dummies(df.debt_median)
new_cash_dummy=pd.get_dummies(df.cash_median)
new_asset_dummy=pd.get_dummies(df.asset_median)
regulation_dummy=pd.get_dummies(df.deregulation,prefix='dereg')
retail_dummy=pd.get_dummies(df.retail_choice,prefix='ret_choice')
year_dummy=pd.get_dummies(df.year,prefix='year')


In [ ]:
df=pd.concat([df,segment_dummy,debt_dummy,cash_dummy,asset_dummy,regulation_dummy,retail_dummy,year_dummy,new_debt_dummy,new_cash_dummy,new_asset_dummy],axis=1)

## Create some more interacting variables

In [ ]:
df['vertical_dereg']=df['vertical']*df['dereg_YES']

In [ ]:
df['post_debt']=df['post'] *df['debt_asset']
df['post_LT_debt']=df['post'] * df['LT_debt_asset']
df['post_four_LT_debt']=df['post'] *df['four_lags_LT_debt_asset']
df['post_ST_debt']=df['post'] * df['ST_debt_asset']
df['post_four_ST_debt']=df['post'] *df['four_lags_ST_debt_asset']
df['post_four_debt']=df['post'] *df['four_lags_debt_asset']
df['post_cash']=df['post'] *df['cash_asset']
df['post_four_cash']=df['post'] *df['four_lags_cash_asset']
df['post_cash_flow']=df['post'] *df['cash_flow_asset']
df['post_retained_earn_asset']=df['post'] *df['retainedearnings_asset']
df['post_external_finance']=df.post*df.external_finance
df['post_highdebt']=df['post']*df['high_debt']
df['post_meddebt']=df['post']*df['medium_debt']
df['post_lowdebt']=df['post']*df['low_debt']


df['post_quarter1']=df['post']*df['quarter_1']
df['post_quarter2']=df['post']*df['quarter_2']
df['post_quarter3']=df['post']*df['quarter_3']
df['post_quarter4']=df['post']*df['quarter_4']


df['post_dereg_YES']=df['post']*df.dereg_YES
df['post_dereg_NO']=df['post']*df.dereg_NO
df['post_dereg_suspended']=df['post']*df.dereg_SUSPENDED
df['post_vertical']=df['post']*df['vertical']
df['post_vertical_dereg']=df['post']* df['vertical']* df['dereg_YES']

df['post_retail_YES']=df['post']*df.ret_choice_YES
df['post_retail_NO']=df['post']*df.ret_choice_NO
df['post_retail_suspended']=df['post']*df.ret_choice_SUSPENDED

df['post_subsid']=df['post']*df['stko']

In [ ]:
df['post_year_debt']=df['post'] *df['year_debt']
df['post_year_LT_debt']=df['post'] * df['year_LT_debt']
df['post_year_ST_debt']=df['post'] * df['year_ST_debt']
df['post_year_cash']=df['post'] *df['year_cash']

In [ ]:
df['post_sept08']=[1 if x>=datetime.datetime(2008,9,1) else 0 for x in df['date']] #lehman brothers collapse
df['firstyear']=[1 if x>datetime.datetime(2007,6,30) and x<=datetime.datetime(2008,6,30) else 0 for x in df['date']]
df['secondyear']=[1 if x>datetime.datetime(2008,6,30) else 0 for x in df['date']]

In [ ]:
df['first_debt']=df['firstyear'] *df['debt_asset']
#df['first_four_debt']=df['firstyear'] *df['four_lags_debt_asset']
df['first_LT_debt']=df['firstyear'] * df['LT_debt_asset']
df['first_four_LT_debt']=df['firstyear'] *df['four_lags_LT_debt_asset']
df['first_ST_debt']=df['firstyear'] * df['ST_debt_asset']
df['first_four_ST_debt']=df['firstyear'] *df['four_lags_ST_debt_asset']
df['first_cash']=df['firstyear'] *df['cash_asset']
df['first_four_cash']=df['firstyear'] *df['four_lags_cash_asset']
df['first_cash_flow']=df['firstyear'] *df['cash_flow_asset']
df['first_retained_earn_asset']=df['firstyear'] *df['retainedearnings_asset']
df['first_external_finance']=df.firstyear *df.external_finance
df['first_highdebt']=df['firstyear']*df['high_debt']
df['first_meddebt']=df['firstyear']*df['medium_debt']
df['first_lowdebt']=df['firstyear']*df['low_debt']

df['first_quarter1']=df['firstyear']*df['quarter_1']
df['first_quarter2']=df['firstyear']*df['quarter_2']
df['first_quarter3']=df['firstyear']*df['quarter_3']
df['first_quarter4']=df['firstyear']*df['quarter_4']


df['first_dereg_YES']=df['firstyear']*df.dereg_YES
df['first_dereg_NO']=df['firstyear']*df.dereg_NO
df['first_dereg_suspended']=df['firstyear']*df.dereg_SUSPENDED
df['first_vertical']=df['firstyear']*df['vertical']
df['first_vertical_dereg']=df['firstyear']* df['vertical']* df['dereg_YES']


df['first_retail_YES']=df['firstyear']*df.ret_choice_YES
df['first_retail_NO']=df['firstyear']*df.ret_choice_NO
df['first_retail_suspended']=df['firstyear']*df.ret_choice_SUSPENDED


df['first_subsid']=df['firstyear']*df['stko']

In [ ]:
df['second_debt']=df['secondyear'] *df['debt_asset']
df['second_LT_debt']=df['secondyear'] * df['LT_debt_asset']
df['second_four_ST_debt']=df['secondyear'] *df['four_lags_ST_debt_asset']
df['second_ST_debt']=df['secondyear'] * df['ST_debt_asset']
df['second_four_LT_debt']=df['secondyear'] *df['four_lags_LT_debt_asset']
df['second_cash']=df['secondyear'] *df['cash_asset']
df['second_four_cash']=df['secondyear'] *df['four_lags_cash_asset']
df['second_cash_flow']=df['secondyear'] *df['cash_flow_asset']
df['second_retained_earn_asset']=df['secondyear'] *df['retainedearnings_asset']
df['second_external_finance']=df.secondyear *df.external_finance
df['second_highdebt']=df['secondyear']*df['high_debt']
df['second_meddebt']=df['secondyear']*df['medium_debt']
df['second_lowdebt']=df['secondyear']*df['low_debt']

df['second_quarter1']=df['secondyear']*df['quarter_1']
df['second_quarter2']=df['secondyear']*df['quarter_2']
df['second_quarter3']=df['secondyear']*df['quarter_3']
df['second_quarter4']=df['secondyear']*df['quarter_4']


df['second_dereg_YES']=df['secondyear']*df.dereg_YES
df['second_dereg_NO']=df['secondyear']*df.dereg_NO
df['second_dereg_suspended']=df['secondyear']*df.dereg_SUSPENDED
df['second_vertical']=df['secondyear']*df['vertical']
df['second_vertical_dereg']=df['secondyear']* df['vertical']* df['dereg_YES']

df['second_retail_YES']=df['secondyear']*df.ret_choice_YES
df['second_retail_NO']=df['secondyear']*df.ret_choice_NO
df['second_retail_suspended']=df['secondyear']*df.ret_choice_SUSPENDED


df['second_subsid']=df['secondyear']*df['stko']

In [ ]:
df['first_year_debt']=df['firstyear'] *df['year_debt']
df['first_year_LT_debt']=df['firstyear'] * df['year_LT_debt']
df['first_year_ST_debt']=df['firstyear'] * df['year_ST_debt']
df['first_year_cash']=df['firstyear'] *df['year_cash']

df['second_year_debt']=df['secondyear'] *df['year_debt']
df['second_year_LT_debt']=df['secondyear'] * df['year_LT_debt']
df['second_year_ST_debt']=df['secondyear'] * df['year_ST_debt']
df['second_year_cash']=df['secondyear'] *df['year_cash']

In [ ]:
dd=[]
for row in df['year']:
    if row==2004:
        dd.append(1)
    elif row==2005:
        dd.append(2)
    elif row==2006:
        dd.append(3)
    elif row==2007:
        dd.append(4)
    elif row==2008:
        dd.append(5)
    elif row==2009:
        dd.append(6)
    elif row==2010:
        dd.append(7)
    elif row==2011:
        dd.append(8)
    else:
        dd.append(0)
df['year_time']=dd

## Firms that have data in both pre and post crisis preiods

In [ ]:
post_crisis_firms=(df[df.date==datetime.datetime(2009,3,31)]['tic'].unique())
pre_crisis_firms=(df[df.post==0]['tic'].unique())

In [ ]:
lasting_firms=list(set(post_crisis_firms) & set(pre_crisis_firms))

In [ ]:
print len(lasting_firms)
print len(post_crisis_firms)
print len(pre_crisis_firms)

In [ ]:
dd=[]
for tic in df['tic']:
    if tic in lasting_firms:
        dd.append(1)
    else:
        dd.append(0)

In [ ]:
df['lasting_firms']=dd

In [ ]:
len(df.tic.unique())

In [ ]:
DF=df[df.remove==1]

In [ ]:
len(DF.tic.unique())

In [ ]:
DF=DF[DF.segment != 'generation'] # remove firms that are only involved in generation

In [ ]:
len(DF.tic.unique())

In [ ]:
DF=DF[(DF.conm !='NIAGARA MOHAWK POWER CORP') & (DF.conm !='MAINE & MARITIMES CORP')]

In [ ]:
len(DF.conm.unique())

In [ ]:
DF['stock_purchase_asset']=(DF['stock_purchase']/DF.atq) * 100

In [ ]:
DF['sale_change']=DF['sale_asset']-DF['four_lags_sale_asset']

## Final sample (from 2004 to 2010)

In [ ]:
dd=[]
for index,row in DF.iterrows():
    if row['fyearq']==2004 and row['fqtr']==1:
        dd.append(1)
    else:
        dd.append(0)
DF['Q12004']=dd        

In [ ]:
DF=DF[DF['Q12004']==0]

In [ ]:
len(DF.tic.unique())

In [ ]:
DF['NWC_asset']=(DF['wcapq']/DF['atq'])*100

In [ ]:
## Operating cash flow

In [ ]:
DF['OCF']=DF['oibdpq']+DF['dpq']-DF['txpq']+DF['wcapq'] #operating income, depreciation, total income tax, working cap changes

In [ ]:
DF['OCF_asset']=(DF['OCF']/DF['atq']) * 100

## Create Lagged Variables

In [ ]:
grouped=DF.groupby(['tic',])
new=[] ## for every tic and fiscal year, 
for group in grouped.groups.keys():
    DFF=grouped.get_group(group)
    DFF['lag_property_sale_asset']=DFF['property_sale_asset'].shift(1)
    DFF['lag_plant_add_asset']=DFF['plant_add_asset'].shift(1) #Utility Plant - Gross Additions
    DFF['lag_stock_sale_asset']=DFF['stock_sale_asset'].shift(1)
    DFF['lag_stock_purchase_asset']=DFF['stock_purchase_asset'].shift(1)
    DFF['lag_investment_sale_asset']=DFF['investment_sale_asset'].shift(1)
    DFF['lag_other_cash_asset']=DFF['other_cash_asset'].shift(1)
    DFF['lag_payout_asset']=DFF['payout_asset'].shift(1)
    DFF['lag_LT_debt_issuance_asset']=DFF['LT_debt_issuance_asset'].shift(1)
    DFF['lag_LT_debt_reduction_asset']=DFF['LT_debt_reduction_asset'].shift(1)
    DFF['lag_NOCF_asset']=DFF['NOCF_asset'].shift(1)
    #DF['net_debt_issuance']=DF['total_debt_q']-DF['total_debt_q'].shift(1)
    new.append(DFF)


In [ ]:
DF=pd.concat(new)

In [ ]:
## Create RTO variable

In [ ]:
dd=[]
for row in DF['state']:
    if row=='CA':
        dd.append('CAISO')
    elif row=='PA' or row=='DE' or row=='IL' or row=='IN' or row=='KY' or row=='MD' or row=='MI' or row=='NJ' or row=='NC' or row=='OH' or row=='TN' or row=='VA' or row=='WV' or row=='DC':
        dd.append('PJM')
    elif row=='TX':
        dd.append('ERCOT')
    else:
        dd.append(None)
DF['RTO']=dd

### Create RRA ratings variable that reflects opinion on Public Utility Commission

In [ ]:
from time import time
start_time=time()
dd=[]
for index,row in DF.iterrows():
    if row['state']=='AL':
        dd.append(2)
    elif row['state']=='AZ' and row['date']>=datetime.datetime(2005,3,1):
        dd.append(6)
    elif row['state']=='AZ' and row['date']<datetime.datetime(2005,3,1):
        dd.append(7)
    elif row['state']=='AR' and row['date']>=datetime.datetime(2006,4,7):
        dd.append(7)
    elif row['state']=='AR' and row['date']<datetime.datetime(2006,4,7):
        dd.append(6)
    elif row['state']=='CA' and row['date']>=datetime.datetime(2007,4,5):
        dd.append(4)
    elif row['state']=='CA' and row['date']>=datetime.datetime(2005,12,29) and row['date']<datetime.datetime(2007,4,5):
        dd.append(5)
    elif row['state']=='CA' and row['date']>=datetime.datetime(2004,4,15) and row['date']<datetime.datetime(2005,12,29):
        dd.append(6)
    elif row['state']=='CA' and row['date']<datetime.datetime(2004,4,15):
        dd.append(7)
    elif row['state']=='CO' and row['date']>=datetime.datetime(2007,8,21):
        dd.append(5)
    elif row['state']=='CO' and row['date']<datetime.datetime(2007,8,21):
        dd.append(6)
    elif row['state']=='CT' and row['date']<datetime.datetime(2008,4,1):
        dd.append(6)
    elif row['state']=='CT' and row['date']>=datetime.datetime(2008,4,1) and row['date']<datetime.datetime(2009,3,23):
        dd.append(7)
    elif row['state']=='CT' and row['date']>=datetime.datetime(2009,3,23) and row['date']<datetime.datetime(2009,7,16):
        dd.append(8)
    elif row['state']=='CT' and row['date']>=datetime.datetime(2009,7,16):
        dd.append(9)
    elif row['state']=='DC':
        dd.append(5)
    elif row['state']=='DE':
        dd.append(4)
    elif row['state']=='HI':
        dd.append(5)
    elif row['state']=='FL' and row['date']>=datetime.datetime(2010,1,13):
        dd.append(4)
    elif row['state']=='FL' and row['date']>=datetime.datetime(2009,10,2) and row['date']<datetime.datetime(2010,1,13):
        dd.append(3)
    elif row['state']=='FL' and row['date']<datetime.datetime(2009,10,2):
        dd.append(2)
    elif row['state']=='GA' and row['date']>=datetime.datetime(2005,10,5):
        dd.append(4)
    elif row['state']=='GA' and row['date']<datetime.datetime(2005,10,5):
        dd.append(5)
    elif row['state']=='ID' and row['date']>=datetime.datetime(2010,8,27):
        dd.append(5)
    elif row['state']=='ID' and row['date']<datetime.datetime(2010,8,27):
        dd.append(6)
    elif row['state']=='IL' and row['date']>=datetime.datetime(2007,4,5):
        dd.append(8)
    elif row['state']=='IL' and row['date']>=datetime.datetime(2005,9,23) and row['date']<datetime.datetime(2007,4,5):
        dd.append(7)
    elif row['state']=='IL' and row['date']<datetime.datetime(2005,9,23):
        dd.append(6)
    elif row['state']=='IN' and row['date']>=datetime.datetime(2009,7,15):
        dd.append(3)
    elif row['state']=='IN' and row['date']<datetime.datetime(2009,7,15):
        dd.append(2)
    elif row['state']=='IA' and row['date']>=datetime.datetime(2007,2,1):
        dd.append(3)
    elif row['state']=='IA' and row['date']<datetime.datetime(2007,2,1):
        dd.append(4)
    elif row['state']=='KS' and row['date']>=datetime.datetime(2008,3,27):
        dd.append(5)
    elif row['state']=='KS' and row['date']<datetime.datetime(2008,3,27):
        dd.append(6)
    elif row['state']=='KY' and row['date']>=datetime.datetime(2010,12,9):
        dd.append(4)
    elif row['state']=='KY' and row['date']>=datetime.datetime(2005,4,1) and row['date']<datetime.datetime(2010,12,9):
        dd.append(5)
    elif row['state']=='KY' and row['date']<datetime.datetime(2005,4,1):
        dd.append(4)
    elif row['state']=='LA' and row['date']>=datetime.datetime(2007,10,16):
        dd.append(5)
    elif row['state']=='LA' and row['date']<datetime.datetime(2007,10,16):
        dd.append(6)
    elif row['state']=='MD' and row['date']<datetime.datetime(2006,4,11):
        dd.append(4)
    elif row['state']=='MD' and row['date']>=datetime.datetime(2006,4,11) and row['date']<datetime.datetime(2008,1,18):
        dd.append(5)
    elif row['state']=='MD' and row['date']>=datetime.datetime(2008,1,18) and row['date']<datetime.datetime(2008,5,12):
        dd.append(6)
    elif row['state']=='MD' and row['date']>=datetime.datetime(2008,5,12) and row['date']<datetime.datetime(2009,7,15):
        dd.append(7)
    elif row['state']=='MD' and row['date']>=datetime.datetime(2009,7,15):
        dd.append(8)
    elif row['state']=='MA':
        dd.append(4)
    elif row['state']=='MI' and row['date']<datetime.datetime(2004,4,12):
        dd.append(4)
    elif row['state']=='MI' and row['date']>=datetime.datetime(2004,4,12) and row['date']<datetime.datetime(2010,4,9):
        dd.append(5)
    elif row['state']=='MI' and row['date']>=datetime.datetime(2010,4,9):
        dd.append(4)
    elif row['state']=='MS' and row['date']<datetime.datetime(2008,7,15):
        dd.append(3)
    elif row['state']=='MS' and row['date']>=datetime.datetime(2008,7,15):
        dd.append(2)
    elif row['state']=='MN':
        dd.append(5)
    elif row['state']=='MO' and row['date']<datetime.datetime(2008,1,8):
        dd.append(6)
    elif row['state']=='MO' and row['date']>=datetime.datetime(2008,1,8):
        dd.append(5)
    elif row['state']=='MT':
        dd.append(7)
    elif row['state']=='NE' and row['date']<datetime.datetime(2004,1,28):
        dd.append(7)
    elif row['state']=='NE' and row['date']>=datetime.datetime(2008,1,28):
        dd.append(5)
    elif row['state']=='NV' and row['date']<datetime.datetime(2006,11,1):
        dd.append(7)
    elif row['state']=='NV' and row['date']>=datetime.datetime(2006,11,1) and row['date']<datetime.datetime(2007,5,23):
        dd.append(6)
    elif row['state']=='NV' and row['date']>=datetime.datetime(2007,5,23):
        dd.append(5)
    elif row['state']=='NH':
        dd.append(6)
    elif row['state']=='NM' and row['date']<datetime.datetime(2004,1,5):
        dd.append(5)
    elif row['state']=='NM' and row['date']>=datetime.datetime(2004,1,5) and row['date']<datetime.datetime(2008,4,1):
        dd.append(6)  
    elif row['state']=='NM' and row['date']>=datetime.datetime(2008,4,1):
        dd.append(7)
    elif row['state']=='NY' and row['date']>=datetime.datetime(2007,10,24):
        dd.append(6)
    elif row['state']=='NY' and row['date']<datetime.datetime(2007,10,24):
        dd.append(5)
    elif row['state']=='OH' and row['date']<datetime.datetime(2009,10,8):
        dd.append(5)
    elif row['state']=='OH' and row['date']>=datetime.datetime(2009,10,8):
        dd.append(4)
    elif row['state']=='OK' and row['date']<datetime.datetime(2007,10,15):
        dd.append(5)
    elif row['state']=='OK' and row['date']>=datetime.datetime(2007,10,15):
        dd.append(6)
    elif row['state']=='OR' and row['date']<datetime.datetime(2004,2,1):
        dd.append(4)
    elif row['state']=='OR' and row['date']>=datetime.datetime(2004,2,1) and row['date']<datetime.datetime(2006,1,6):
        dd.append(6)
    elif row['state']=='OR' and row['date']>=datetime.datetime(2006,1,6) and row['date']<datetime.datetime(2006,9,22):
        dd.append(5)
    elif row['state']=='OR' and row['date']>=datetime.datetime(2006,9,22):
        dd.append(6)
    elif row['state']=='PA':
        dd.append(6)
    elif row['state']=='SC':
        dd.append(4)
    elif row['state']=='SD':
        dd.append(5)
    elif row['state']=='TN':
        dd.append(4)
    elif row['state']=='TX':
        dd.append(7)
    elif row['state']=='WI':
        dd.append(2)
    elif row['state']=='WV'and row['date']>=datetime.datetime(2008,3,28):
        dd.append(6)
    elif row['state']=='WV'and row['date']<datetime.datetime(2008,3,28):
        dd.append(7)
    elif row['state']=='NJ'and row['date']>=datetime.datetime(2004,5,12):
        dd.append(5)
    elif row['state']=='NJ'and row['date']<datetime.datetime(2004,5,12):
        dd.append(4)
    elif row['state']=='NC':
        dd.append(5)
    elif row['state']=='VA':
        dd.append(3)
    elif row['state']=='VT' and row['date']<datetime.datetime(2005,4,27):
        dd.append(6)
    elif row['state']=='VT' and row['date']>=datetime.datetime(2005,4,27) and row['date']<datetime.datetime(2007,4,1):
        dd.append(7)
    elif row['state']=='VT' and row['date']>=datetime.datetime(2007,4,1):
        dd.append(6)
    elif row['state']=='WA' and row['date']<datetime.datetime(2008,1,8):
        dd.append(4)
    elif row['state']=='WA' and row['date']>=datetime.datetime(2008,1,8) and row['date']<datetime.datetime(2010,4,9):
        dd.append(5)
    elif row['state']=='WA' and row['date']>=datetime.datetime(2010,4,9):
        dd.append(6)
    elif row['state']=='WI':
        dd.append(2)
    else:
        dd.append(None)
end_time=time()
 
 
        

In [ ]:
DF['RRA']=dd

In [ ]:
DF['stock_payout']=DF['stock_sale_asset']*DF['payout_asset']

In [ ]:
DF.to_csv('.../data/sample')